**2. Data preprocessing**
*   The Daraengineering library contains functions such as encode function that performs label encoding and PCA on the input data.

Import libraries

In [ ]:
import Dataengineering as De
import tensorflow as tf
import tempfile
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
import shap

c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
genotype_data = pd.read_csv('Depression sum data.csv')
genotype_data

,rsid,genotype,effect weight,P,Case of disease
0,rs189107123,CG,-0.121998,0.11030,0
1,rs180734498,TC,0.038903,0.16240,0
2,rs144762171,CG,0.040902,0.40540,0
3,chr1_13957_D,ID,-0.067904,0.38910,0
4,rs151276478,TC,-0.109603,0.07397,0
...,...,...,...,...,...
9874284,rs191117135,AG,-0.078805,0.33760,0
9874285,chr22_51236013_I,ID,-0.005696,0.75450,0
9874286,rs3896457,TC,-0.033898,0.01707,0
9874287,rs149733995,AC,-0.008597,0.72340,0


In [ ]:
df=De.encode(genotype_data)
df


,rsid,genotype,effect weight,P,Case of disease,genotype_encoded,rsid_encoded,pca_encoded_genotype_1,pca_encoded_genotype_2
0,rs189107123,CG,-0.121998,0.11030,0,3,4701041,2.361030e+05,-0.483004
1,rs180734498,TC,0.038903,0.16240,0,6,4380931,5.562130e+05,2.496270
2,rs144762171,CG,0.040902,0.40540,0,3,3564786,1.372358e+06,-0.556575
3,chr1_13957_D,ID,-0.067904,0.38910,0,5,320423,4.616721e+06,1.233355
4,rs151276478,TC,-0.109603,0.07397,0,6,3989152,9.479920e+05,2.470902
...,...,...,...,...,...,...,...,...,...
9874284,rs191117135,AG,-0.078805,0.33760,0,1,4777736,1.594080e+05,-2.478038
9874285,chr22_51236013_I,ID,-0.005696,0.75450,0,5,418648,4.518496e+06,1.239715
9874286,rs3896457,TC,-0.033898,0.01707,0,6,5977094,-1.039950e+06,2.599620
9874287,rs149733995,AC,-0.008597,0.72340,0,0,3888328,1.048816e+06,-3.535626


In [ ]:
df=pd.read_csv('Despression summ.csv')
X = df[['effect weight', 'P', 'genotype_encoded', 'rsid_encoded', 'pca_encoded_genotype_1', 'pca_encoded_genotype_2']]
y = df['Case of disease']


Feature selection helps improve model performance by reducing dimensionality and removing irrelevant or redundant features. SelectKBest selects the top k features based on a statistical test. It’s using the f_classif (ANOVA F-value) test to evaluate the importance of each feature.

In [ ]:
X_new = SelectKBest(f_classif, k=4).fit_transform(X, y)
X_new

array([[-1.21998157e-01,  1.10300000e-01,  4.70104100e+06,
         2.36102998e+05],
       [ 3.89033551e-02,  1.62400000e-01,  4.38093100e+06,
         5.56213000e+05],
       [ 4.09019913e-02,  4.05400000e-01,  3.56478600e+06,
         1.37235800e+06],
       ...,
       [-3.38981034e-02,  1.70700000e-02,  5.97709400e+06,
        -1.03995000e+06],
       [-8.59684723e-03,  7.23400000e-01,  3.88832800e+06,
         1.04881600e+06],
       [-2.48973836e-02,  2.37200000e-01,  4.42218500e+06,
         5.14959000e+05]])

In [ ]:
X = df[['effect weight', 'P', 'rsid_encoded', 'pca_encoded_genotype_1']]
y = df['Case of disease']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Scaling the data is a recommended pre-processing step when working with deep learning neural networks. It helps ensure stable learning and better model performance. Using the famous method StandardScaler, It standardizes features by removing the mean and scaling to unit variance.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled=scaler.transform(X)


**3. Model Training and Hyperparameters**

In [ ]:
from scikeras.wrappers import KerasClassifier

# The function to create the model
def create_model(input_shape, optimizer='adam', activation='relu', units=16):
    model = Sequential([
        Dense(32, input_shape=input_shape, activation=activation),
        Dense(units, activation=activation),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model
# Create a KerasClassifier wrapper
model = KerasClassifier(build_fn=create_model, input_shape=X_train_scaled.shape[1:], verbose=0)

# Define hyperparameters to search
param_grid = {
    'batch_size': [8, 16],
    'epochs': [30, 50],
    'optimizer': ['adam', 'sgd'],
    'model__activation': ['relu', 'tanh'],
    'model__units': [16, 32]  # Use model__units instead of units
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Get the best hyperparameters
best_params = grid_result.best_params_
print(f"Best hyperparameters: {best_params}")



c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\P

c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\P

Best hyperparameters: {'batch_size': 8, 'epochs': 30, 'model__activation': 'relu', 'model__units': 16, 'optimizer': 'adam'}


'\'\n# Train the model with the best hyperparameters\nbest_model = create_model(**best_params)\nhistory = best_model.fit(X_train_scaled, y_train, epochs=best_params[\'epochs\'], batch_size=best_params[\'batch_size\'], verbose=1, validation_split=0.2)\n\n# Evaluate the model\nloss, accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=0)\nprint(f"Model accuracy: {accuracy * 100:.2f}%")'

Training the model with the best paramaters

In [ ]:
best_model = create_model(X_train_scaled.shape[1:], optimizer=best_params['optimizer'], activation=best_params['model__activation'], units=best_params['model__units'])
best_model.compile(optimizer=best_params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])
best_batch_size = best_params['batch_size']
best_epochs = best_params['epochs']
best_model.fit(X_test_scaled, y_test, batch_size=best_batch_size, epochs=best_epochs, verbose=1)


c:\Users\Moussa Benoumhani\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 316s 1ms/step - accuracy: 0.9915 - loss: 0.0214
Epoch 2/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 198s 800us/step - accuracy: 0.9945 - loss: 0.0118
Epoch 3/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 204s 827us/step - accuracy: 0.9954 - loss: 0.0103
Epoch 4/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 316s 1ms/step - accuracy: 0.9957 - loss: 0.0094
Epoch 5/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 222s 899us/step - accuracy: 0.9961 - loss: 0.0088
Epoch 6/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 186s 752us/step - accuracy: 0.9964 - loss: 0.0082
Epoch 7/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 173s 702us/step - accuracy: 0.9966 - loss: 0.0078
Epoch 8/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 188s 760us/step - accuracy: 0.9967 - loss: 0.0076
Epoch 9/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 387s 2ms/step - accuracy: 0.9969 - loss: 0.0073
Epoch 10/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 841s 3ms/step - accuracy: 0.9969 - loss: 0.0071
Epoch 11/30
246858/246858 ━━━━━━━━━━━━━━━━━━━━ 

This code computes SHAP values for each feature in the dataset and then combines them to calculate the importance of each row. The resulting mean_values provide insights into which rows contribute most significantly to the model’s predictions.

In [ ]:
explainer = shap.KernelExplainer(best_model.predict, X_scale)
# Get the SHAP values for all the data
shap_values = explainer.shap_values(X_scaled, nsamples=100)
sh=np.array(shap_values)
# Extract and print SHAP values for each row
df_array=np.hstack((sh, np.expand_dims(sh.mean(axis=1), axis=1)))
# Calculate the mean value of the shap values for each row
mean_values = np.mean(df_array, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step


  0%|          | 0/9 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step 


 11%|█         | 1/9 [00:00<00:06,  1.17it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


...

 78%|███████▊  | 7/9 [00:03<00:00,  2.24it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


 89%|████████▉ | 8/9 [00:04<00:00,  2.11it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


100%|██████████| 9/9 [00:04<00:00,  2.01it/s]


Add a new column to the dataset that contains the mean_values, which represent the importance for each row. We name this column ‘new weights’ because we’ll use it in the calculation of the PRS (Partial Regression Score) instead of the effect weight

In [ ]:
df['New weights']=mean_values
df.to_csv('Despression.csv', index=False)

In [ ]:
df.head()

,rsid,genotype,effect weight,P,Case of disease,genotype_encoded,rsid_encoded,pca_encoded_genotype_1,pca_encoded_genotype_2,New weights
1,rs180734498,TC,0.038903,0.16240,0,6,4380931,5.562130e+05,2.496270,-0.003666
2,rs144762171,CG,0.040902,0.40540,0,3,3564786,1.372358e+06,-0.556575,0.018471
3,chr1_13957_D,ID,-0.067904,0.38910,0,5,320423,4.616721e+06,1.233355,-0.039316
4,rs151276478,TC,-0.109603,0.07397,0,6,3989152,9.479920e+05,2.470902,0.037904
5,rs140337953,TG,-0.023504,0.25150,0,7,3276732,1.660412e+06,3.424774,-0.007108
